Imports

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import re
import json
import fitz  
from PIL import Image
from io import BytesIO
from urllib.parse import urlparse
import spacy
import google.generativeai as genai
from dotenv import load_dotenv
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")

/Users/ziyaozhou/Desktop/XR_Lab_Autograder/venv311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configure Gemini

In [2]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)
text_model = genai.GenerativeModel("gemini-2.0-flash")
vision_model = genai.GenerativeModel("gemini-2.0-flash")

Rubric

In [3]:
rubric = {
    "architect_chosen": 5,
    "doc_and_slides": 5,
    "bio_750_words": 5,
    "bio_structure": 5,
    "bio_references": 5,
    "10_buildings_with_images": 5,
    "image_quality": 5,
    "image_citations": 5,
    "image_relevance": 5,
    "personal_bio_photo": 5,
    "presentation_polish": 5
}

rubric_descriptions = {
    "architect_chosen": "Is the architect selected from Book Two and clearly identified?",
    "doc_and_slides": "Is the document structured well with table of contents and all required sections?",
    "bio_750_words": "Does the biography meet the 750-word requirement?",
    "bio_structure": "Does the biography cover who they are, where they studied, etc.?",
    "bio_references": "Are there 5–10 APA references with DOIs and citation counts?",
    "10_buildings_with_images": "Are 10 buildings covered with names, locations, significance, and image suggestions?",
    "image_quality": "Are the images high-resolution and well-composed?",
    "image_citations": "Do all images have proper attribution (photographer/source)?",
    "image_relevance": "Do images clearly relate to the architect’s work?",
    "personal_bio_photo": "Is a professional student photo and 1–2 sentence bio included?",
    "presentation_polish": "Is the document polished, well-formatted, and web-publishable?"
}

 Extract text from PDF

In [4]:
pdf_path = "/Users/ziyaozhou/desktop/submissions/dasilvatheo_LATE_171930_14930244_HW A1.pdf"

In [5]:
def extract_text_from_pdf(pdf_path):
    print(f" Extracting text from: {pdf_path}")
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    print(" Extracted text from PDF")
    return text

 Extract images from PDF

In [6]:
def extract_images_from_pdf(pdf_path, min_width=1200, save_folder="/Users/ziyaozhou/Desktop/XR_Lab/Extracted_images"):
    print(f" Extracting images from: {pdf_path}")
    doc = fitz.open(pdf_path)
    os.makedirs(save_folder, exist_ok=True)
    image_data = []
    for page_index in range(len(doc)):
        page = doc[page_index]
        images = page.get_images(full=True)
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            img_pil = Image.open(BytesIO(image_bytes))
            width, height = img_pil.size
            img_pil.save(os.path.join(save_folder, f"page{page_index+1}_img{img_index+1}.png"))
            image_data.append({
                "page": page_index + 1,
                "width": width,
                "height": height,
                "coordinates": img[1:5],
                "image": img_pil,
                "filename": f"page{page_index+1}_img{img_index+1}.png",
                "is_high_res": width >= min_width
            })
    print(f" Extracted {len(image_data)} images")
    return image_data

Match Image with Nearby Text

In [7]:
def get_caption_candidates(text, image_data):
    print("Scanning for image captions...")
    lines = text.split("\n")
    results = []

    for img in image_data:
        context = {
            "page": img["page"],
            "image": img["filename"],
            "matched_caption": "",
            "has_citation": False,
            "has_building_name": False,
            "has_interior_note": False
        }
        for i, line in enumerate(lines):
            if f"{img['filename'].split('.')[0]}" in line:
                nearby_lines = lines[max(i-2, 0): i+3]
                caption_text = " ".join(nearby_lines)
                context["matched_caption"] = caption_text
                context["has_citation"] = any(x in caption_text.lower() for x in ["source", "http", "photographer"])
                context["has_building_name"] = bool(re.search(r"(building|tower|museum|villa|house|center)", caption_text, re.IGNORECASE))
                context["has_interior_note"] = bool(re.search(r"(interior|lobby|hall|inside)", caption_text, re.IGNORECASE))
                break
        results.append(context)
    return results

Evaluate Image Relevance, Building Match, and Attribution via Gemini

In [8]:
def evaluate_images_with_gemini(image_data, architect_name, debug=False):
    print(" Evaluating image content and relevance using Gemini...")
    enriched_image_feedback = []

    for img in tqdm(image_data, desc="Evaluating images"):
        prompt = f"""
You are reviewing an image submitted for a university architecture project about {architect_name}.
Please analyze this image and answer:

1. Does this image show a building designed by {architect_name}? If yes, specify the building.
2. Is this an interior or exterior shot?
3. Does this image clearly show architectural features (e.g., lighting, geometry, layout)?
4. How relevant is this image for an academic project about {architect_name}?

Give your feedback in the following JSON format:
{{
  "building_detected": "...",
  "interior_or_exterior": "...",
  "relevance_score": "x/10",
  "justification": "...",
  "architectural_features_visible": true/false
}}
"""
        try:
            response = vision_model.generate_content([img["image"], prompt])
            if debug:
                print(f"Image {img['filename']} feedback:\n", response.text)
            cleaned_text = response.text.strip()
            if cleaned_text.startswith("```"):
                cleaned_text = re.sub(r"```(?:json)?", "", cleaned_text)
                cleaned_text = cleaned_text.replace("```", "").strip()

            try:
                data = json.loads(cleaned_text)
            except Exception as e:
                print(f" Still failed to parse JSON from {img['filename']}: {e}")
                data = {
                    "building_detected": "Unknown",
                    "interior_or_exterior": "Unknown",
                    "relevance_score": "5/10",
                    "justification": "Could not parse feedback from Gemini.",
                    "architectural_features_visible": False
                }
        except Exception as e:
            print(f"⚠️ Error processing {img['filename']}: {e}")
            data = {
                "building_detected": "Unknown",
                "interior_or_exterior": "Unknown",
                "relevance_score": "5/10",
                "justification": "Could not extract structured feedback.",
                "architectural_features_visible": False
            }

        data.update({
            "filename": img["filename"],
            "page": img["page"],
            "width": img["width"],
            "height": img["height"],
            "is_high_res": img["is_high_res"]
        })
        enriched_image_feedback.append(data)
    return enriched_image_feedback

Score Image Citations and Structure Compliance

In [9]:
def evaluate_image_structure_and_captions(image_feedback, caption_context):
    print("Evaluating caption presence and structure")

    scores = []
    per_image_feedback = []

    for img in image_feedback:
        caption = next((c for c in caption_context if c["image"] == img["filename"]), {})
        has_citation = caption.get("has_citation", False)
        has_building_name = caption.get("has_building_name", False)
        has_interior_note = caption.get("has_interior_note", False)

        score = 0
        if has_citation: score += 3
        if has_building_name: score += 3
        if has_interior_note: score += 2
        if img["is_high_res"]: score += 2

        scores.append(score)
        per_image_feedback.append({
            "image": img["filename"],
            "page": img["page"],
            "relevance_score": img.get("relevance_score", "5/10"),
            "justification": img.get("justification", ""),
            "caption_found": caption.get("matched_caption", ""),
            "has_proper_caption": score >= 7,
            "score": score
        })

    avg_score = sum(scores) / len(scores) if scores else 0
    return {
        "score": int((avg_score / 10) * rubric["image_citations"]),
        "details": per_image_feedback
    }

Gemini-Based Rubric Evaluation with Chain-of-Thought

In [10]:
def gemini_detailed_rubric_eval(text, architect_name):
    print(" Gemini evaluating full rubric with explanations")

    prompt = f"""
You are evaluating a student's architecture assignment on the architect {architect_name}.

This is a formal submission for university credit. You are receiving the full document as **images**, so you can directly observe the formatting, embedded images, captions, structure, and layout.

---

###  How to Grade:

- Be **fair and constructive**. If formatting is inconsistent, information is missing, or citations are weak, **please call it out clearly**.
- Do not sugarcoat — students are expected to revise based on your feedback.
- If something is strong, note it. If it's flawed, critique it.
- When scoring, **prioritize**:
  - Accuracy of academic citations
  - Caption and image attribution clarity
  - Clear distinction between interior vs exterior images
  - Overall layout and visual professionalism

---

###  Additional Clarifications:

-  Images are embedded (not just links)
-  Captions below images include attribution (URLs or photographer names)
-  A student photo and bio appear on Page 2
-  Table of Contents is present
-  10 buildings are described
-  Redundant links are likely citations, not missing content

---

###  RUBRIC CRITERIA

Please assess each of the following categories. For every criterion:

1. Give a **detailed justification** (1–2 paragraphs)
2. Assign a score **out of 5** based on the detailed rubric below

Format:
**[Category Name]**
Justification: ...
Score: x/5

---

###  Categories and Rubric Anchors

**1. Architect Selection & Scope**
- 5 = Clearly identifies one architect from Book Two, explicitly stated, on-topic
- 3–4 = Identifies Book Two architect, but clarity or justification could improve
- 1–2 = Architect unclear, off-topic, or not from Book Two

**2. Organization & Document Setup**
- 5 = Clear Table of Contents + labeled sections for bio, buildings, refs, student bio
- 3–4 = Minor issues with layout or missing headers
- 1–2 = Poor organization, missing sections, or hard to follow

**3. Biographical Content (750 words)**
- 5 = Covers who they are, achievements, education, significance, 1st building, typologies
- 3–4 = Mostly complete, with slight omissions or light detail
- 1–2 = Underdeveloped or below word count, missing major points

**4. Citation of Architect Biography**
- 5 = 5–10 academic references, correct APA formatting, includes DOIs and citation counts
- 3–4 = APA errors or missing DOIs, but still academically relevant
- 1–2 = Few or no academic references, poor or irrelevant sources

**5. Selection & Quality of Images**
- 5 = 10 buildings, 3+ exterior + 5+ interior per building, high-res
- 3–4 = Most buildings meet criteria; a few lack resolution or quantity
- 1–2 = Many buildings missing images or poor quality

**6. Image Citation & Attribution**
- 5 = Every image has clear, consistent source or photographer citation
- 3–4 = Most are cited but with some inconsistencies
- 1–2 = Citations mostly missing, inconsistent, or improperly formatted

**7. Coverage of 10 Famous Buildings**
- 5 = All 10 named + location + significance statement (1–2 sentences)
- 3–4 = Buildings listed but some lack significance or location
- 1–2 = Several missing or incomplete building descriptions

**8. Image Relevance**
- 5 = All images relate directly to described buildings, match descriptions, show architectural value
- 3–4 = Most images relevant, some generic or misaligned
- 1–2 = Several images are off-topic or not associated with described buildings

**9. Personal Bio & Photo**
- 5 = Professional photo and bio (1–2 sentences), correctly placed after TOC
- 3–4 = Present but minor formatting/image issues
- 1–2 = Photo or bio is low quality, misplaced, or absent

**10. Overall Completeness & Presentation**
- 5 = Fully polished, clean layout, minimal repetition, suitable for web/publication
- 3–4 = Clear submission, but lacks design polish or has formatting repetition
- 1–2 = Sloppy or rushed presentation; visual issues hurt readability

---

 Please start your rubric-based analysis below:
"""

    doc = fitz.open(pdf_path)
    all_pages_as_images = [page.get_pixmap(dpi=300).pil_tobytes("png") for page in doc]
    response = vision_model.generate_content(
         [prompt] + [Image.open(BytesIO(img_bytes)) for img_bytes in all_pages_as_images]
      )

    print(response.text)

    def extract_score(label, out_of):
        match = re.search(label + r".*?Score:\s*(\d+)/" + str(out_of), response.text, re.IGNORECASE | re.DOTALL)
        return int(match.group(1)) if match else 0

    return {
        "architect_chosen": {"score": extract_score("Architect Selection", 5)},
        "doc_and_slides": {"score": extract_score("Organization", 5)},
        "bio_750_words": {"score": extract_score("Biographical Content", 5)},
        "bio_structure": {"score": extract_score("Biographical Structure", 5)},
        "bio_references": {"score": extract_score("Citation of Architect Biography", 5)},
        "10_buildings_with_images": {"score": extract_score("Coverage of 10 Famous Buildings", 5)},
        "image_quality": {"score": extract_score("Selection & Quality of Images", 5)},
        "image_citations": {"score": extract_score("Image Citation & Attribution", 5)},
        "image_relevance": {"score": extract_score("Image Relevance", 5)},
        "personal_bio_photo": {"score": extract_score("Personal Bio", 5)},
        "presentation_polish": {"score": extract_score("Presentation Polish", 5)},
    }

Final Aggregator + Full Diagnostic Report

In [11]:
# def generate_detailed_scorecard(scores, image_caption_details=None):
#     print(" Compiling final scorecard")

#     # Total and max only for defined rubric keys
#     total = sum([scores[k]["score"] for k in scores if k in rubric])
#     max_total = sum([rubric[k] for k in scores if k in rubric])
#     final_percentage = (total / max_total) * 100 if max_total else 0

#     grade = "A" if final_percentage >= 50 else "B" if final_percentage >= 46 else "C" if final_percentage >= 42 else "D"

#     # print(f"Final Grade: {grade} ({round(final_percentage, 2)}%)")
#     rubric_table = pd.DataFrame([
#         {
#             "Criterion": k.replace("_", " ").title(),
#             "Score": scores[k]["score"],
#             "Max": rubric[k],
#             "Description": rubric_descriptions.get(k, "")
#         }
#         for k in rubric if k in scores
#     ])
#     display(rubric_table)
#     if image_caption_details:
#         print("\n Image Caption & Relevance Feedback:")
#         df = pd.DataFrame(image_caption_details["details"])
#         display(df)

#     return {
#         "rubric_scores": {k: scores[k]["score"] for k in rubric if k in scores},
#         "final_percent": round(final_percentage, 2),
#         "grade": grade,
#         "image_feedback_table": image_caption_details
#     }

def generate_detailed_scorecard(scores, image_caption_details=None):
    print(" Compiling final scorecard")

    # Total and max only for defined rubric keys
    total = sum([scores[k]["score"] for k in scores if k in rubric])
    max_total = sum([rubric[k] for k in scores if k in rubric])
    final_percentage = (total / max_total) * 100 if max_total else 0

    grade = "A" if final_percentage >= 50 else "B" if final_percentage >= 46 else "C" if final_percentage >= 42 else "D"

    # print(f"Final Grade: {grade} ({round(final_percentage, 2)}%)")
    rubric_table = pd.DataFrame([
        {
            "Criterion": k.replace("_", " ").title(),
            "Score": scores[k]["score"],
            "Max": rubric[k],
            "Description": rubric_descriptions.get(k, "")
        }
        for k in rubric if k in scores
    ])
    return rubric_table

In [12]:
def extract_references_from_text(text):
    print(" Extracting references from text")
    lines = text.split("\n")
    references = []
    for line in lines:
        if re.search(r"\(\d{4}\)", line) and any(x in line.lower() for x in ["doi", "archdaily", "e-architect", "https://", "http://"]):
            references.append(line.strip())
    return references


In [13]:
def evaluate_biography(text):
    print(" Evaluating biography: checking word count and required sections")
    result = {}
    doc = nlp(text)
    result["word_count"] = len([token.text for token in doc if token.is_alpha])

    required_sections = [
        "who they are",
        "famous for",
        "studied",
        "significance",
        "influence",
        "types of buildings",
        "first building"
    ]

    section_hits = sum([1 for section in required_sections if section.lower() in text.lower()])
    result["structure_score"] = int((section_hits / len(required_sections)) * rubric["bio_structure"])
    result["score"] = rubric["bio_750_words"] if result["word_count"] >= 700 else int((result["word_count"] / 750) * rubric["bio_750_words"])

    return result


In [14]:
def evaluate_image_quality(image_data):
    print(" Evaluating image resolution")
    high_res_count = sum(1 for img in image_data if img["is_high_res"])
    total_images = len(image_data)
    
    quality_score = int((high_res_count / max(1, total_images)) * rubric["image_quality"])
    
    print(f" {high_res_count}/{total_images} images are high resolution")
    return {"high_res_count": high_res_count, "score": quality_score}

Main Runner Pipeline

In [18]:
# def run_autograder_full(pdf_path, architect_name="Bjarke Ingels", debug=False):
#     print("Starting full grading pipeline")
#     text = extract_text_from_pdf(pdf_path)
#     images = extract_images_from_pdf(pdf_path)
#     references = extract_references_from_text(text)
#     image_caption_context = get_caption_candidates(text, images)
#     enriched_feedback = evaluate_images_with_gemini(images, architect_name, debug)
#     image_caption_score = evaluate_image_structure_and_captions(enriched_feedback, image_caption_context)
#     gemini_scores = gemini_detailed_rubric_eval(text, architect_name)
#     manual_scores = {}
#     renamed_gemini = {
#         "architect_chosen": gemini_scores["architect_chosen"],
#         "doc_and_slides": gemini_scores["doc_and_slides"],
#         "personal_bio_photo": gemini_scores["personal_bio_photo"],
#         "presentation_polish": gemini_scores["presentation_polish"]
#     }
#     all_scores = {**manual_scores, **renamed_gemini}
#     result = generate_detailed_scorecard(all_scores, image_caption_score)
#     return result

def run_autograder_full(pdf_path, architect_name="Bjarke Ingels", debug=False):
    print("Starting full grading pipeline")
    text = extract_text_from_pdf(pdf_path)
    images = extract_images_from_pdf(pdf_path)
    references = extract_references_from_text(text)
    image_caption_context = get_caption_candidates(text, images)
    enriched_feedback = evaluate_images_with_gemini(images, architect_name, debug)
    image_caption_score = evaluate_image_structure_and_captions(enriched_feedback, image_caption_context)
    gemini_scores = gemini_detailed_rubric_eval(text, architect_name)
    manual_scores = {}
    renamed_gemini = {
        "architect_chosen":           gemini_scores["architect_chosen"],
        "doc_and_slides":             gemini_scores["doc_and_slides"],
        "bio_750_words":              gemini_scores["bio_750_words"],
        "bio_structure":              gemini_scores["bio_structure"],
        "bio_references":             gemini_scores["bio_references"],
        "10_buildings_with_images":   gemini_scores["10_buildings_with_images"],
        "image_quality":              gemini_scores["image_quality"],
        "image_citations":            gemini_scores["image_citations"],
        "image_relevance":            gemini_scores["image_relevance"],
        "personal_bio_photo":         gemini_scores["personal_bio_photo"],
        "presentation_polish":        gemini_scores["presentation_polish"],
    }
    all_scores = {**manual_scores, **renamed_gemini}
    result = generate_detailed_scorecard(all_scores, image_caption_score)
    return result

Run 

In [19]:
result = run_autograder_full("/Users/ziyaozhou/desktop/submissions/dasilvatheo_LATE_171930_14930244_HW A1.pdf", "Bjarke Ingels", debug=True)

Starting full grading pipeline
 Extracting text from: /Users/ziyaozhou/desktop/submissions/dasilvatheo_LATE_171930_14930244_HW A1.pdf
 Extracted text from PDF
 Extracting images from: /Users/ziyaozhou/desktop/submissions/dasilvatheo_LATE_171930_14930244_HW A1.pdf
 Extracted 76 images
 Extracting references from text
Scanning for image captions...
 Evaluating image content and relevance using Gemini...


Evaluating images:   1%|▏         | 1/76 [00:02<03:11,  2.55s/it]

Image page1_img1.png feedback:
 ```json
{
  "building_detected": "No",
  "interior_or_exterior": "N/A",
  "relevance_score": "1/10",
  "justification": "The image shows a person wearing a graduation cap and gown, likely related to a graduation ceremony. It does not depict any architectural structure, let alone one designed by Bjarke Ingels. It is highly irrelevant to an architecture project about Bjarke Ingels.",
  "architectural_features_visible": false
}
```


Evaluating images:   3%|▎         | 2/76 [00:04<02:55,  2.37s/it]

Image page8_img1.png feedback:
 ```json
{
  "building_detected": "No",
  "interior_or_exterior": "Interior",
  "relevance_score": "1/10",
  "justification": "The image shows the interior of the Cathedral of Brasília, designed by Oscar Niemeyer, not Bjarke Ingels. Thus, it is not relevant to a project about Bjarke Ingels' architecture. Architectural features are clearly visible (roof structure, layout, sculptures), but they're relevant to Niemeyer, not Ingels.",
  "architectural_features_visible": true
}
```


...

Image page22_img2.png feedback:
 {"building_detected": "No",
  "interior_or_exterior": "Interior",
  "relevance_score": "1/10",
  "justification": "This image shows the interior of a legislative building that is not designed by Bjarke Ingels. It appears to be the National Congress of Brazil in Brasilia. The relevance to an academic project about Bjarke Ingels is very low unless it's used as a contrast to his style.",
  "architectural_features_visible": true
}


Evaluating images:  41%|████      | 31/76 [01:17<02:00,  2.69s/it]

Image page23_img1.png feedback:
 ```json
{
  "building_detected": "Niteroi Contemporary Art Museum (MAC) by Oscar Niemeyer",
  "interior_or_exterior": "exterior",
  "relevance_score": "1/10",
  "justification": "The image shows the Niteroi Contemporary Art Museum, which was designed by Oscar Niemeyer, not Bjarke Ingels. Therefore, it is not relevant for an architecture project about Bjarke Ingels.",
  "architectural_features_visible": true
}
```


Evaluating images:  41%|████      | 31/76 [01:19<01:54,  2.55s/it]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from pathlib import Path

# 1. Your mapping of PDF filenames to chosen architects
pdf_dict = {
    "dasilvatheo_LATE_171930_14930244_HW A1.pdf": "Oscar Niemeyer",
    "davidmatthew_LATE_134808_14949557_COGS 160_ A1.pdf": "Kazuyo Sejima",
    "delacruzrenier_LATE_226065_14930691_Paul Rudolph_ Life, Work, and Enduring Influence.pdf": "Paul Rudolph",
    "emralinolalaine_LATE_162831_14938886_Glenn Murcutt_ “Touching the Earth Lightly”.pdf": "Glenn Murcutt",
    "hsucalvin_166834_14967682_Cogs 160 Slide and Doc Links for Turn-in.pdf": "Frank Lloyd Wright",
    "khirwadkarisha_166304_14925482_Report.pdf": "Ken Yeang",
    "krukjulia_LATE_198551_15046680_Eero Saarinen_ Cogs 160 Research Document.pdf": "Eero Saarinen",
    "liangmichael_188529_14924464_COGS 160 - Docs.pdf": "Zaha Hadid",
    "mainayardaniel_127050_14924649_COGS 160 Le Corbusier Doc.pdf": "Le Corbusier",
    "marvanalicia_212624_14925657_A.Marvan_LuisBarragan.pdf": "Luis Barragan",
    "spavenchristine_LATE_96300_14929508_COGS 160_1 (1).pdf": "Richard Rogers",
    "vidyalatanvi_LATE_218146_14937211_COGS 160 A1-1.docx": "Norman Robert Foster",
    "wucynthia_LATE_167097_15019933_COGS 160.pdf": "Philip Johnson",
    "yangheiman_LATE_190478_14963531_COGS160.pdf": "Sou Fujimoto",
}

def batch_scorecards(pdf_dict, submissions_folder, debug=False):
    submissions_folder = Path(submissions_folder)
    records = []

    for filename, architect in pdf_dict.items():
        pdf_path = submissions_folder / filename
        if not pdf_path.exists():
            print(f"⚠️ Warning: file not found: {pdf_path}")
            continue

        rubric_df = run_autograder_full(str(pdf_path), architect, debug=debug)
        rubric_df = rubric_df.assign(File=filename, Architect=architect)
        records.append(rubric_df)

    long_df = pd.concat(records, ignore_index=True)
    return long_df

submissions_dir = "/Users/ziyaozhou/desktop/submissions"
long_df = batch_scorecards(pdf_dict, submissions_dir, debug=False)
print(long_df.head())

wide_df = long_df.pivot_table(
    index=["File", "Architect"],
    columns="Criterion",
    values="Score"
).reset_index()

wide_df.columns.name = None
print(wide_df.head())


Starting full grading pipeline
 Extracting text from: /Users/ziyaozhou/desktop/submissions/dasilvatheo_LATE_171930_14930244_HW A1.pdf
 Extracted text from PDF
 Extracting images from: /Users/ziyaozhou/desktop/submissions/dasilvatheo_LATE_171930_14930244_HW A1.pdf
 Extracted 76 images
 Extracting references from text
Scanning for image captions...
 Evaluating image content and relevance using Gemini...


Evaluating images:  13%|█▎        | 10/76 [00:19<02:20,  2.13s/it]

 Still failed to parse JSON from page12_img2.png: Expecting value: line 1 column 1 (char 0)


Evaluating images:  49%|████▊     | 37/76 [01:27<01:48,  2.79s/it]

 Still failed to parse JSON from page26_img1.png: Expecting value: line 1 column 1 (char 0)


...

 Still failed to parse JSON from page42_img2.png: Expecting value: line 1 column 1 (char 0)


Evaluating images:  98%|█████████▊| 79/81 [03:45<00:05,  2.82s/it]

 Still failed to parse JSON from page43_img1.png: Expecting value: line 1 column 1 (char 0)


Evaluating images: 100%|██████████| 81/81 [03:50<00:00,  2.85s/it]


Evaluating caption presence and structure
 Gemini evaluating full rubric with explanations
Okay, I will evaluate the student's submission based on the provided rubric and images.

**1. Architect Selection & Scope**
Justification: The student chose Oscar Niemeyer, who is a well-known architect. Assuming this architect is included in "Book Two" (which I cannot verify), the selection is appropriate and clearly stated.
Score: 5/5

**2. Organization & Document Setup**
Justification: The document is *partially* organized. There is a rudimentary list of buildings, and a section for the student bio. However, there is no Table of Contents, and there are no clear headers for the biography, building descriptions, or reference sections *in the main body*. The student uses titles such as "Step 1: Personal Information," which is an odd, informal approach. This needs a formal Table of Contents (which is missing) and better section labeling.
Score: 2/5

**3. Biographical Content (750 words)**
Justific

In [ ]:
import pandas as pd

def compute_weights_by_variance(long_df, score_col="Score", criterion_col="Criterion"):
    var_series = long_df.groupby(criterion_col)[score_col].var(ddof=0)
    variance_df = var_series.reset_index(name="Variance")
    total_var = variance_df["Variance"].sum()
    if total_var == 0:
        variance_df["Weight"] = 1.0 / len(variance_df)
    else:
        variance_df["Weight"] = variance_df["Variance"] / total_var
    return variance_df.sort_values("Variance", ascending=False).reset_index(drop=True)

def apply_weighted_grades(wide_df, variance_df):
    weight_map = dict(zip(variance_df["Criterion"], variance_df["Weight"]))
    criterion_cols = [c for c in wide_df.columns if c not in ["File", "Architect"]]
    wide_df["WeightedScore"] = 0.0
    for c in criterion_cols:
        weight = weight_map.get(c, 0.0)
        wide_df["WeightedScore"] += (wide_df[c] / 5.0) * weight
    wide_df["WeightedScorePercent"] = wide_df["WeightedScore"] * 100
    return wide_df

variance_df = compute_weights_by_variance(long_df)
wide_df = apply_weighted_grades(wide_df, variance_df)

print(variance_df)
print(wide_df[["File", "Architect", "WeightedScorePercent"]])
wide_df.to_csv('wide_df.csv', index=False)

criterion_cols = [c for c in wide_df.columns if c not in ["File", "Architect", "WeightedScore", "WeightedScorePercent"]]
wide_df['RawScorePercent'] = wide_df[criterion_cols].sum(axis=1) / (5 * len(criterion_cols)) * 100

wide_df['RawRank']      = wide_df['RawScorePercent'].rank(ascending=False, method='dense').astype(int)
wide_df['WeightedRank'] = wide_df['WeightedScorePercent'].rank(ascending=False, method='dense').astype(int)

print(wide_df[['File','Architect','RawScorePercent','RawRank','WeightedScorePercent','WeightedRank']])

wide_df.to_csv('wide_df_with_raw_and_weighted_ranks.csv', index=False)

                   Criterion  Variance    Weight
0              Bio 750 Words  0.698225  0.196013
1         Personal Bio Photo  0.686391  0.192691
2             Doc And Slides  0.556213  0.156146
3             Bio References  0.390533  0.109635
4            Image Citations  0.378698  0.106312
5   10 Buildings With Images  0.366864  0.102990
6              Image Quality  0.236686  0.066445
7            Image Relevance  0.177515  0.049834
8           Architect Chosen  0.071006  0.019934
9              Bio Structure  0.000000  0.000000
10       Presentation Polish  0.000000  0.000000
                                                 File           Architect  \
0          dasilvatheo_LATE_171930_14930244_HW A1.pdf      Oscar Niemeyer   
1   davidmatthew_LATE_134808_14949557_COGS 160_ A1...       Kazuyo Sejima   
2   delacruzrenier_LATE_226065_14930691_Paul Rudol...        Paul Rudolph   
3   emralinolalaine_LATE_162831_14938886_Glenn Mur...       Glenn Murcutt   
4   hsucalvin_166834_149676